In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import pandas as pd
from finbert_embedding.embedding import FinbertEmbedding
from transformers import BertTokenizer, BertForSequenceClassification
import numpy as np
from tqdm import tqdm
import glob
import os.path
from pandas import *
from collections import Counter
import time

In [2]:
current_data_directory = os.path.abspath("./data/sp500")
path_text = current_data_directory + "/text/"
path_to_id_csv = current_data_directory + "/df_100stocks_sp500.csv"


In [3]:
print(current_data_directory)

/Users/juliencyrusenyegue/Documents/GitHub/financial_econometrics_project/FinBert/data/sp500


In [4]:
data = read_csv(path_to_id_csv)
ID_sp500 = data['id'].tolist()

In [5]:
alphabets= "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov)"

In [6]:
finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone',num_labels=3)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')

In [7]:
# Extract texts from the directory

def extract_texts(start, N):

    texts = []
    first_sentence = []
    articles = []
    list_ID = []
   
    list_tickers = os.listdir(path_text)
    list_tickers.remove('.DS_Store') # for mac
 
    for ticker in list_tickers:
        
        list_articles = glob.glob(path_text+ticker+"/*")
        
        ID = []
        for s in list_articles:
            
            test_id = int(s.replace(path_text+str(ticker)+"/", ""))
          
            if test_id in ID_sp500:
                
                with open(s) as f:
                    
                    x = re.sub(path_text+str(ticker)+"/","",s)
                    articles.append(x)
                    t = f.read()
                    texts.append(t)
        
                ID.append(test_id)
                  
        list_ID += ID
        
     
    return texts, list_ID


In [8]:
def split_into_sentences(text):
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]

    #sentences = [x for x in sentences if len(x.split(' '))>5]
    return sentences


In [9]:
# Run finbert on each text and count the nb of neutral, positive, negative sentences

def finbert_texts(start,n_texts, texts):
    labels = {0:'neutral', 1:'positive',2:'negative'}
    data = np.zeros((n_texts,3))

    for j in tqdm(range(start,start + n_texts)):
        
        print(j - start + 1)
        sentences = split_into_sentences(texts[j])
        sentences = texts[j].split('\n \n')
        sentences = [x for x in sentences if len(x.split(' '))>4]
        inputs = tokenizer(sentences, return_tensors="pt", padding=True,truncation=True,max_length=512)


        outputs = finbert(**inputs)[0]
        #print(outputs)
        i = np.argmax(outputs.detach().numpy(), axis = 1)
        #print(i)
        sentiment_counter = list(Counter(i).values())
        if(np.shape(sentiment_counter) == (2, )):
            sentiment_counter.append(0)
        print(sentiment_counter)
        data[j-start,:] = sentiment_counter
        time.sleep(45)
        
        
    return data

In [10]:
# Nombre d'articles
N = 50
start = 200
texts, list_ID = extract_texts(start, N)


li = [1,2]
#print(np.shape(li))

if(np.shape(li) == (2, )):
    li.append(4)
    print(li)
    

In [11]:
 if __name__ == "__main__":
    
    data = finbert_texts(start, N, texts)
    
   

  0%|                                                    | 0/50 [00:00<?, ?it/s]

1
[90, 26, 9]


  2%|▊                                        | 1/50 [01:41<1:22:45, 101.35s/it]

2
[109, 24, 9]


  4%|█▋                                        | 2/50 [03:15<1:17:47, 97.24s/it]

...

 96%|███████████████████████████████████████▎ | 48/50 [1:07:43<02:20, 70.35s/it]

49
[81, 37, 1]


 98%|████████████████████████████████████████▏| 49/50 [1:09:14<01:16, 76.72s/it]

50
[55, 22, 3]


100%|█████████████████████████████████████████| 50/50 [1:10:42<00:00, 84.86s/it]


In [12]:
# Ratios computations

df_data = pd.DataFrame(data, columns= ["Neutral","Positive", "Negative"])


df_data["Total"] = df_data["Neutral"] + df_data["Positive"] + df_data["Negative"]
df_data["Difference"] = df_data["Positive"] - df_data["Negative"]
df_data["P_N ratio"] = df_data["Positive"] / df_data["Negative"]
df_data["Difference ratio"] = df_data["Difference"] / df_data["Total"]

df_data["Decile"] = 1 + df_data["Difference ratio"].transform(lambda y: pd.qcut(y, 10, labels=False))

df_data = np.round(df_data, 5)


df_data["ID"] = list_ID[start: start + N]
first_column = df_data.pop("ID")
df_data.insert(0, "ID", first_column)

#long = list(df_data.loc[df_data['Decile'].isin([10,9])]["ID"])
#short = list(df_data.loc[df_data['Decile'].isin([1,2])]["ID"])

display(df_data)


,ID,Neutral,Positive,Negative,Total,Difference,P_N ratio,Difference ratio,Decile
0,4419494,90.0,26.0,9.0,125.0,17.0,2.88889,0.13600,2
1,4358685,109.0,24.0,9.0,142.0,15.0,2.66667,0.10563,1
2,4460641,53.0,28.0,6.0,87.0,22.0,4.66667,0.25287,5
3,4255002,60.0,25.0,9.0,94.0,16.0,2.77778,0.17021,3
4,4480173,111.0,28.0,12.0,151.0,16.0,2.33333,0.10596,1
5,4317441,51.0,16.0,3.0,70.0,13.0,5.33333,0.18571,3
6,4439385,53.0,33.0,3.0,89.0,30.0,11.00000,0.33708,7
7,4379483,90.0,45.0,21.0,156.0,24.0,2.14286,0.15385,2
8,4378855,25.0,28.0,2.0,55.0,26.0,14.00000,0.47273,9
9,4439214,27.0,41.0,1.0,69.0,40.0,41.00000,0.57971,10


In [13]:
df_ratios = read_csv(os.path.abspath("./FinBert_ratios.csv"))
display(df_ratios)
df_ratios = pd.concat([df_ratios, df_data])
df_ratios["Decile"] = 1 + df_ratios["Difference ratio"].transform(lambda y: pd.qcut(y, 10, labels=False))
display(df_ratios)

,ID,Neutral,Positive,Negative,Total,Difference,P_N ratio,Difference ratio,Decile
0,4463311,32.0,34.0,4.0,70.0,30.0,8.50000,0.42857,10
1,4320409,54.0,35.0,5.0,94.0,30.0,7.00000,0.31915,8
2,4256968,44.0,5.0,37.0,86.0,-32.0,0.13514,-0.37209,1
3,4442839,25.0,6.0,29.0,60.0,-23.0,0.20690,-0.38333,1
4,4298941,43.0,30.0,6.0,79.0,24.0,5.00000,0.30380,8
...,...,...,...,...,...,...,...,...,...
195,4501965,62.0,13.0,1.0,76.0,12.0,13.00000,0.15789,5
196,4296861,64.0,3.0,21.0,88.0,-18.0,0.14286,-0.20455,1
197,4275572,62.0,11.0,2.0,75.0,9.0,5.50000,0.12000,4
198,4338042,57.0,9.0,1.0,67.0,8.0,9.00000,0.11940,4


,ID,Neutral,Positive,Negative,Total,Difference,P_N ratio,Difference ratio,Decile
0,4463311,32.0,34.0,4.0,70.0,30.0,8.50000,0.42857,10
1,4320409,54.0,35.0,5.0,94.0,30.0,7.00000,0.31915,8
2,4256968,44.0,5.0,37.0,86.0,-32.0,0.13514,-0.37209,1
3,4442839,25.0,6.0,29.0,60.0,-23.0,0.20690,-0.38333,1
4,4298941,43.0,30.0,6.0,79.0,24.0,5.00000,0.30380,8
...,...,...,...,...,...,...,...,...,...
45,4327174,37.0,15.0,4.0,56.0,11.0,3.75000,0.19643,5
46,4346324,32.0,13.0,1.0,46.0,12.0,13.00000,0.26087,7
47,4360233,35.0,18.0,5.0,58.0,13.0,3.60000,0.22414,6
48,4504519,81.0,37.0,1.0,119.0,36.0,37.00000,0.30252,8


In [14]:
df_ratios.to_csv('FinBert_ratios.csv', index = False)